In [6]:
import math
#%tensorflow_version 1.x
import tensorflow as tf
import numpy as np
import json
import tensorflow_datasets as tfds
import matplotlib.pyplot as plt
from tensorflow.keras.applications.resnet_v2 import preprocess_input
from tensorflow.keras.applications.resnet_v2 import decode_predictions
from IPython.display import Image, display
from IPython.display import Image, display
import keras

In [2]:
%cd /content/drive/MyDrive/cnn_embed/cnn_manifold

/content/drive/MyDrive/cnn_embed/cnn_manifold


In [29]:
!git config --global user.email "aliciaxiaozeng@gmail.com"
!git config --global user.name "alicialitrtwe"

In [14]:
!git add .

fatal: not a git repository (or any of the parent directories): .git


In [15]:
!git commit -m 'visualize for any direction'

fatal: not a git repository (or any of the parent directories): .git


In [23]:
# TODO proper reception field size
# TODO image initiation does not work for inception deeper layers
# TODO optimization steps

In [7]:
layer_name = "conv3_block4_out"
model = tf.keras.applications.ResNet50V2(weights="imagenet", include_top=False)
#layer_name = "mixed3"
#model = tf.keras.applications.InceptionV3(weights="imagenet", include_top=False)
layer = model.get_layer(name=layer_name)
activation_model = tf.keras.models.Model(inputs=model.inputs, outputs=layer.output)


94674944/94668760 [==============================] - 1s 0us/step


In [8]:
def initialize_image(img_size):
    # We start from a gray image with some random noise
    img = tf.Variable(tf.random.normal([1, img_size, img_size, 3], 0, 0.01, tf.float32, seed=1))
    # ResNet50V2 expects inputs in the range [-1, +1].
    return img

def compute_loss(img, direction, activation_model, img_size=180, cos_sim=False, x=None, y=None):
    #TODO: batch
    acts = activation_model(img) 
    # visualize the neuron in the center of each layer
    shape = tf.shape(acts)
    x_ = shape[1] // 2 if x is None else x
    y_ = shape[2] // 2 if y is None else y
    filter_activation = acts[0, 2:-2, 2:-2, :]
    # TODO: cosine similarity
    if cos_sim==False:
        loss = -tf.reduce_mean(filter_activation*direction)
    return loss
def gradient_ascent_step(img, direction, activation_model, learning_rate, opt):
    with tf.GradientTape() as tape:
        tape.watch(img)
        loss = compute_loss(img, direction, activation_model)
    # Compute gradients.
    grads = tape.gradient(loss, img)
    # Normalize gradients.
    grads = tf.math.l2_normalize(grads)
    opt.apply_gradients(zip([grads], [img]))
    loss = compute_loss(img, direction, activation_model)
    return loss, img
def visualize_filter(img, direction, activation_model, img_size=180, learning_rate=0.1, n_iter=100):
    
    # gradient ascent
    #img = initialize_image(img_size)
    opt = tf.keras.optimizers.Adam(learning_rate=learning_rate)
    for i in range(n_iter):
        loss, img = gradient_ascent_step(img, direction, activation_model, learning_rate, opt)
        # Progress
        if (i % 100 == 0):
            print(f'i = {i}')
    # Decode the resulting input image
    processed_img = deprocess_image(img[0].numpy())
    return loss, img, processed_img
def deprocess_image(img):
    # Normalize array: center on 0., ensure variance is 0.15
    img -= img.mean()
    img /= img.std() + 1e-5
    img *= 0.15

    # Center crop
    img = img[25:-25, 25:-25, :]

    # Clip to [0, 1]
    img += 0.5
    img = np.clip(img, 0, 1)

    # Convert to RGB array
    img *= 255
    img = np.clip(img, 0, 255).astype("uint8")
    return img

In [9]:
img_size = 180

In [10]:
directions = np.random.random((4, 512))

In [13]:
%%time
loss_list = []
visualization_list = []
for i in range(len(directions)):
    img = initialize_image(img_size)
    loss, img, processed_img = visualize_filter(img, directions[i], activation_model, n_iter=4, learning_rate=100)
    loss_list.append(loss)
    visualization_list.append(img)

i = 0
i = 0
i = 0
i = 0
CPU times: user 982 ms, sys: 7.49 ms, total: 989 ms
Wall time: 987 ms


In [ ]:
keras.preprocessing.image.save_img("0.png", processed_img)
display(Image("0.png"))

In [ ]:
all_imgs = []
for filter_index in range(64):
    print("Processing filter %d" % (filter_index,))
    loss, img = visualize_filter(filter_index)
    all_imgs.append(img)

# Build a black picture with enough space for
# our 8 x 8 filters of size 128 x 128, with a 5px margin in between
margin = 5
n = 8
cropped_width = img_width - 25 * 2
cropped_height = img_height - 25 * 2
width = n * cropped_width + (n - 1) * margin
height = n * cropped_height + (n - 1) * margin
stitched_filters = np.zeros((width, height, 3))

# Fill the picture with our saved filters
for i in range(n):
    for j in range(n):
        img = all_imgs[i * n + j]
        stitched_filters[
            (cropped_width + margin) * i : (cropped_width + margin) * i + cropped_width,
            (cropped_height + margin) * j : (cropped_height + margin) * j
            + cropped_height,
            :,
        ] = img
keras.preprocessing.image.save_img("stiched_filters.png", stitched_filters)

from IPython.display import Image, display

display(Image("stiched_filters.png"))